In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Pinecone (vector db, latest official package)
!pip install --upgrade pinecone

In [2]:
!pip install --upgrade "langchain>=0.2.0" langchain-openai langchain-pinecone


  Using cached langchain_openai-0.3.28-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_pinecone-0.2.11-py3-none-any.whl.metadata (6.1 kB)
  Using cached langchain_tests-0.3.20-py3-none-any.whl.metadata (3.3 kB)
  Using cached pytest_asyncio-0.26.0-py3-none-any.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 12.7 MB/s eta 0:00:00


In [3]:
!pip install --upgrade langchain-experimental


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [4]:
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5s6oq9f6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5s6oq9f6
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━

In [18]:
!pip install --upgrade yt-dlp


In [ ]:
!pip install tqdm pydub


In [ ]:
!pip install --upgrade numpy


In [7]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")
os.environ["PINECONE_ENVIRONMENT"] = "us-east1-aws"

In [3]:
import yt_dlp
import whisper
import os
from urllib.parse import urlparse, parse_qs

def get_youtube_video_id(url):
    # Handles standard YouTube URLs
    query = urlparse(url)
    if query.hostname in ['www.youtube.com', 'youtube.com']:
        return parse_qs(query.query)['v'][0]
    elif query.hostname == 'youtu.be':
        return query.path[1:]
    else:
        raise ValueError("Invalid YouTube URL")

def download_and_transcribe_youtube(youtube_url, output_dir="/content/drive/MyDrive/github/youtube-rag/rag"):
    video_id = get_youtube_video_id(youtube_url)
    output_filename = os.path.join(output_dir, f'audio_{video_id}.%(ext)s')
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_filename,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'keepvideo': False,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    audio_path = os.path.join(output_dir, f"audio_{video_id}.mp3")

    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    transcript = result['text']

    transcript_path = os.path.join(output_dir, f"transcript_{video_id}.txt")
    with open(transcript_path, "w") as f:
        f.write(transcript)
    return transcript, transcript_path


ModuleNotFoundError: No module named 'yt_dlp'

In [12]:
youtube_url = "https://www.youtube.com/watch?v=HgcoFVqG0ms"
transcript, transcript_path = download_and_transcribe_youtube(youtube_url)
print(f"Transcript saved at: {transcript_path}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=HgcoFVqG0ms
[youtube] HgcoFVqG0ms: Downloading webpage
[youtube] HgcoFVqG0ms: Downloading tv client config
[youtube] HgcoFVqG0ms: Downloading player 6074d4a4-main
[youtube] HgcoFVqG0ms: Downloading tv player API JSON
[youtube] HgcoFVqG0ms: Downloading ios player API JSON
[youtube] HgcoFVqG0ms: Downloading m3u8 information
[info] HgcoFVqG0ms: Downloading 1 format(s): 251-9
[download] Destination: /content/drive/MyDrive/github/youtube-rag/rag/audio_HgcoFVqG0ms.webm
[download] 100% of   17.36MiB in 00:00:00 at 28.79MiB/s  
[ExtractAudio] Destination: /content/drive/MyDrive/github/youtube-rag/rag/audio_HgcoFVqG0ms.mp3
Deleting original file /content/drive/MyDrive/github/youtube-rag/rag/audio_HgcoFVqG0ms.webm (pass -k to keep)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 169MiB/s]


Transcript saved at: /content/drive/MyDrive/github/youtube-rag/rag/transcript_HgcoFVqG0ms.txt


In [8]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings


In [9]:
with open("/content/drive/MyDrive/github/youtube-rag/rag/transcript_HgcoFVqG0ms.txt") as f:
    text = f.read()


In [10]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(),                  # uses your OpenAI key automatically
    breakpoint_threshold_type="percentile",    # can also be 'standard_deviation', 'interquartile', or 'gradient'
    breakpoint_threshold_amount=95.0           # default is 95, tune this percentile if you want more/fewer splits
)


In [11]:
docs = text_splitter.create_documents([text])
print(f"Total semantic chunks: {len(docs)}")
for i, d in enumerate(docs):
    print(f"\n--- Chunk {i+1} ---\n{d.page_content[:300]}")  # Show first 300 chars for preview


Total semantic chunks: 13

--- Chunk 1 ---
 Hello everyone, this video is to explain about Lang graph. Already we discussed about Lang chain. We used Lang chain in A agent that example for storing the context. Similarly, we used Lang chain for our rag project also, right? We used Lang chain. Lang chain is open source software, right? So, tha

--- Chunk 2 ---
Right? So in case of simple agents, it will be like first week for particular task. Okay, but in case of agent K.A, we can just fix a goal like set the goal. And that multi agent systems with that agent K concept, they will be able to like work on achieving that goal. Like by creating multiple agent

--- Chunk 3 ---
Okay. Here in the long-chain, they provided that sample note here. I will be opening it here. So here, the note is like the graph, knowledge graph. It is like notes and edges. Notes are like these commonants, right. Maybe persons, maybe places, okay. They are called as notes generally, okay. But in 

--- Chunk 4 ---
Let

In [12]:
import os
from pinecone import Pinecone, ServerlessSpec

index_name = "youtube-rag-index"

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,        # 1536 for OpenAI Embeddings
        metric='cosine',       # Or 'euclidean', but cosine is default for OpenAI
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )


In [13]:
from langchain_pinecone import Pinecone as LangChainPinecone
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
vectorstore = LangChainPinecone.from_documents(
    docs,
    embeddings,
    index_name=index_name
)
print("Semantic chunks embedded and uploaded to Pinecone vector index!")


Semantic chunks embedded and uploaded to Pinecone vector index!


In [14]:
retriever = vectorstore.as_retriever()


In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="gpt-4.1"  # or "gpt-4o" etc.
)


In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser
# Define prompt for the LLM.
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are an expert assistant. Using ONLY the provided context, answer the user's question.
     ALWAYS respond in this JSON format:
     {{
       "answer": (string, best answer to the question from context),
       "citations": (array of strings, exact text of the context/chunks used)
     }}

     Context:
     {context}"""),
    ("human", "{question}")
])


# Build the chain
rag_chain = (
    RunnableMap({
        "context": retriever,
        "question": RunnablePassthrough()
    })  # This builds {"context": retriever(query), "question": query}
    | prompt
    | llm
    | JsonOutputParser()  # Get just the text out if you want
)


In [23]:
query = "What is a LangGraph or an agent according to the video?"
answer = rag_chain.invoke(query)
print("Answer:", answer)


Answer: {'answer': "According to the video, LangGraph is open source software provided by the same company as LangChain and is designed specifically for implementing multi-agent systems. Unlike LangChain, which is suitable for simple and linear workflows, LangGraph is useful for workflows involving multiple agents, node-like knowledge graphs, and complex relations. A key feature is its capability to easily implement 'human-in-the-loop' concepts, such as including human feedback in workflows. In a multi-agent system, agents can be coordinated to achieve a set goal while maintaining memory and state, with LangGraph providing tools and classes to handle different types of memory (short-term, long-term, episodic) and maintain state across agents.", 'citations': ["Lang chain is for like workflow. It's like kind of linear things, right? One by one, like for very simple workflow, Lang chain is useful. In case of Lang graph, it is for multi agent system. Okay, multi agent system involving like